In [1]:
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
import json
import numpy as np


# df = pd.read_csv("new.csv")

/home/imbalet/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [68]:
with open("new.json") as f:
    df = pd.DataFrame(json.load(f))

In [64]:
with open("regs_new.json", encoding = 'utf-8') as f:
    regs = json.load(f) 

In [69]:
df["region"] = df["region"].apply(lambda x: regs.get(x["id"], ["", "None"])[1])

In [70]:
df["region"] = df["region"].apply(lambda x: x.split(',')[0])

In [76]:
df["body_type_type"] = df["body_type"].apply(lambda x: x["type"])
df["body_type_name"] = df["body_type"].apply(lambda x: x["name"])
df.drop("body_type", inplace=True, axis = 1)

In [77]:
df["super_gen_name"] = df["super_gen"].apply(lambda x: x["name"])
df["super_gen_year_from"] = df["super_gen"].apply(lambda x: x["from"])
df["super_gen_year_to"] = df["super_gen"].apply(lambda x: x["to"])
df.drop("super_gen", inplace=True, axis = 1)

In [78]:
df = df.drop_duplicates()

In [79]:
df.columns

Index(['owners', 'year', 'price', 'region', 'mileage', 'mark', 'model',
       'complectation', 'steering_wheel', 'gear_type', 'engine',
       'transmission', 'power', 'displacement', 'characteristics', 'color',
       'body_type_type', 'body_type_name', 'super_gen_name',
       'super_gen_year_from', 'super_gen_year_to'],
      dtype='object')

In [83]:
df = df[df["owners"] != "None"]
df = df[df["region"] != "None"]
df = df[df["model"] != "None"]

In [84]:
for i in df.columns:
    print(i, len(df[df[i] == "None"]))

owners 0
year 0
price 0
region 0
mileage 0
mark 0
model 0
complectation 269410
steering_wheel 0
gear_type 0
engine 0
transmission 0
power 0
displacement 2869
characteristics 0
color 0
body_type_type 0
body_type_name 0
super_gen_name 46060
super_gen_year_from 0
super_gen_year_to 10698


In [86]:
len(df)

445436

# amomalies


In [87]:

indexes = []
for mark in set(df["mark"]):
    d_m = df[(df["mark"] == mark)]
    for model in set(d_m["model"]):
        d_mm = d_m[d_m["model"] == model]
        for gen in set(d_mm["super_gen_name"]):
            data = d_mm[d_mm["super_gen_name"] == gen]
            print(f"{mark}, {model}, {gen}")
            Q1 = data['price'].quantile(0.25)
            Q3 = data['price'].quantile(0.75)
            IQR = Q3 - Q1
            anomalies = data[(data['price'] < (Q1 - 1.5 * IQR)) | (data['price'] > (Q3 + 1.5 * IQR))]
            indexes.extend(list(anomalies.index))
            



CHERY, Arrizo 5 GT, None
CHERY, Tiggo 5x, I Рестайлинг 3
CHERY, Tiggo 5x, I Рестайлинг
CHERY, Tiggo 5x, I Рестайлинг 2
CHERY, Tiggo 3xe, None
CHERY, Tiggo 3, I
CHERY, Explore 06, None
CHERY, Amulet (A15), I
CHERY, Amulet (A15), I Рестайлинг
CHERY, Tiggo 7 Pro Max, I
CHERY, Tiggo 7 Pro Max, I Рестайлинг
CHERY, M11 (A3), None
CHERY, Tiggo 8 Plus, I
CHERY, Tiggo 8 Plus, I Рестайлинг
CHERY, Arrizo 7, None
CHERY, Tiggo 7 Plus, I
CHERY, Tiggo 7 Plus, I Рестайлинг
CHERY, IndiS (S18D), I
CHERY, Arrizo 5 Plus, None
CHERY, Tiggo 7, I
CHERY, Tiggo 7, II
CHERY, Tiggo 7, II Рестайлинг
CHERY, CrossEastar (B14), None
CHERY, Bonus 3 (E3/A19), None
CHERY, Very (A13), None
CHERY, Tiggo 8, I
CHERY, Tiggo 8, I Рестайлинг
CHERY, Tiggo 2, I
CHERY, Tiggo 2 Pro, None
CHERY, Tiggo 5, I
CHERY, Tiggo 5, I Рестайлинг
CHERY, Tiggo 9, I (China Market)
CHERY, Sweet (QQ), None
CHERY, Tiggo 7 Pro, I
CHERY, Tiggo 4, I
CHERY, Tiggo 4, I Рестайлинг
CHERY, Tiggo 4, I Рестайлинг 2
CHERY, Tiggo 8 Pro e+, None
CHERY, Tiggo 7

In [88]:
len(indexes)

11740

In [89]:
df_cleared = df.drop(indexes)

In [90]:
print(len(df))
print(len(df_cleared))
df_cleared.to_csv('dataset.csv', index=False)

445436
433696


In [98]:
df_cleared.dtypes

owners                 object
year                    int64
price                   int64
region                 object
mileage                 int64
mark                   object
model                  object
complectation          object
steering_wheel         object
gear_type              object
engine                 object
transmission           object
power                  object
displacement           object
characteristics        object
color                  object
body_type_type         object
body_type_name         object
super_gen_name         object
super_gen_year_from    object
super_gen_year_to      object
dtype: object

In [91]:
df_cleared_nona = df_cleared.dropna()